In [103]:
import statsmodels.api as sm
import numpy as np
from econml.dml import LinearDML
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from xgboost import XGBClassifier, XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.neural_network import MLPClassifier, MLPRegressor
from catboost import CatBoostRegressor, CatBoostClassifier
from scipy.stats import logistic

MC_N = 1
MC_θ = np.zeros((MC_N,4))
MC_y = np.zeros((MC_N,4))
MC_t = np.zeros((MC_N,4))

for j in range(MC_N):
    import wooldridge
    df = wooldridge.data('jtrain2')
    #df['avg'] = 0.5 * (df.re74+df.re75)
    df = df.dropna()
    #df = df[df.avg <= 15]
    y = df['re78']
    t = df['train']
    x = df[['age', 'educ', 'black', 'hisp', 'married', 're74', 're75']]
    
    # OLS - Full Estimation
    model_OLS = sm.OLS(y, sm.add_constant(np.c_[t,x]))
    res = model_OLS.fit()
    θ_OLS = res.params[1]
    # OLS First Stage: Y
    model_OLS = sm.OLS(y, sm.add_constant(np.c_[x]))
    res_y = model_OLS.fit()
    θ_OLS_y = res_y.params[1]    
    # Logistic First Stage
    clf = LogisticRegression().fit(x, t)
    
    # DML Lasso
    model_Lasso = LinearDML(discrete_treatment=True, random_state=1, cv=2)
    model_Lasso.fit(y.ravel(), t.ravel(), X=None,W=x)
    θ_DMLL = model_Lasso.intercept_

    # DML RF
    model_XGB = LinearDML(discrete_treatment=True, cv=2,
                          model_y = CatBoostRegressor(),
                          model_t = CatBoostClassifier())
    model_XGB.fit(y.ravel(), t.ravel(), X=None,W=x)
    θ_DMLRF = model_XGB.intercept_
    
    # DML NN - First Stage
    model_NN = LinearDML(discrete_treatment=False, cv=2,
                         model_y = MLPRegressor(random_state=1,
                                                 hidden_layer_sizes=(100), 
                                                 batch_size = x.shape[0],
                                                 momentum=0.95, 
                                                 max_iter=5000, 
                                                 learning_rate_init=0.01, 
                                                 verbose=False, early_stopping=True), 
                         model_t = MLPClassifier(random_state=1,
                                                 hidden_layer_sizes=(100), 
                                                 batch_size = x.shape[0],
                                                 momentum = 0.95, 
                                                 max_iter=5000, 
                                                 learning_rate_init=0.01, 
                                                 verbose=False, early_stopping=True))
    model_NN.fit(y.ravel(), t.ravel(), X=None,W=x)
    θ_DMLRF = model_NN.intercept_

    
    import numpy as np
    from catboost import CatBoostClassifier, Pool, metrics
    from sklearn.model_selection import train_test_split
    x_train, x_test, y_train, y_test = train_test_split(x, t, test_size=0.3)
    print(x_train.shape, y_train.shape, x_test.shape, y_test.shape) 
    train_data = x_train
    train_labels = y_train
    test_data = catboost_pool = Pool(x_test,y_test)
    model_cat_t = CatBoostClassifier()
    model_NN_t = MLPClassifier(max_iter = 10000)
    model_cat_t.fit(x_train, y_train, eval_set = test_data, use_best_model=True)
    model_NN_t.fit(x_train, y_train)
    c = model_cat_t.score(x_test, y_test)
    d = model_NN_t.score(x_test, y_test)
    
    import numpy as np
    from catboost import CatBoostClassifier, Pool, metrics
    from sklearn.model_selection import train_test_split
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)
    print(x_train.shape, y_train.shape, x_test.shape, y_test.shape) 
    train_data = x_train
    train_labels = y_train
    test_data = catboost_pool = Pool(x_test,y_test)
    model_cat_y = CatBoostRegressor()
    model_NN_y = MLPRegressor(max_iter = 10000)
    model_cat_y.fit(x_train, y_train, eval_set = test_data, use_best_model=True)
    model_NN_y.fit(x_train, y_train)
    a = model_cat_y.score(x_test, y_test)
    b = model_NN_y.score(x_test, y_test)

    MC_θ[j] = [θ_OLS, model_Lasso.intercept_, model_XGB.intercept_, model_NN.intercept_]
    MC_y[j] = [res_y.rsquared, np.mean(model_Lasso.nuisance_scores_y), a, b]
    MC_t[j] = [clf.score(x,t), np.mean(model_Lasso.nuisance_scores_t), c, d]

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


Learning rate set to 0.005418
0:	learn: 0.6923132	total: 1.8ms	remaining: 1.8s
1:	learn: 0.6912709	total: 2.55ms	remaining: 1.27s
2:	learn: 0.6906482	total: 3.26ms	remaining: 1.08s
3:	learn: 0.6898315	total: 3.92ms	remaining: 976ms
4:	learn: 0.6893437	total: 4.72ms	remaining: 938ms
5:	learn: 0.6888398	total: 5.38ms	remaining: 891ms
6:	learn: 0.6882529	total: 6.09ms	remaining: 864ms
7:	learn: 0.6877747	total: 7ms	remaining: 868ms
8:	learn: 0.6872599	total: 7.91ms	remaining: 871ms
9:	learn: 0.6865704	total: 8.76ms	remaining: 868ms
10:	learn: 0.6860378	total: 9.62ms	remaining: 865ms
11:	learn: 0.6853853	total: 10.3ms	remaining: 851ms
12:	learn: 0.6846776	total: 11.1ms	remaining: 843ms
13:	learn: 0.6840113	total: 11.8ms	remaining: 829ms
14:	learn: 0.6837208	total: 12.3ms	remaining: 806ms
15:	learn: 0.6833615	total: 13ms	remaining: 797ms
16:	learn: 0.6827508	total: 13.9ms	remaining: 802ms
17:	learn: 0.6821781	total: 14.8ms	remaining: 810ms
18:	learn: 0.6818175	total: 15.4ms	remaining: 797ms

238:	learn: 0.5989696	total: 176ms	remaining: 559ms
239:	learn: 0.5986283	total: 177ms	remaining: 560ms
240:	learn: 0.5982354	total: 178ms	remaining: 560ms
241:	learn: 0.5979048	total: 179ms	remaining: 560ms
242:	learn: 0.5977000	total: 180ms	remaining: 560ms
243:	learn: 0.5973973	total: 180ms	remaining: 559ms
244:	learn: 0.5970613	total: 181ms	remaining: 558ms
245:	learn: 0.5967500	total: 182ms	remaining: 558ms
246:	learn: 0.5964823	total: 183ms	remaining: 558ms
247:	learn: 0.5963904	total: 184ms	remaining: 557ms
248:	learn: 0.5962402	total: 184ms	remaining: 556ms
249:	learn: 0.5959827	total: 185ms	remaining: 555ms
250:	learn: 0.5956431	total: 186ms	remaining: 555ms
251:	learn: 0.5952468	total: 187ms	remaining: 554ms
252:	learn: 0.5949458	total: 187ms	remaining: 553ms
253:	learn: 0.5948385	total: 188ms	remaining: 553ms
254:	learn: 0.5948063	total: 189ms	remaining: 552ms
255:	learn: 0.5946344	total: 190ms	remaining: 552ms
256:	learn: 0.5944794	total: 191ms	remaining: 552ms
257:	learn: 

564:	learn: 0.5342829	total: 526ms	remaining: 405ms
565:	learn: 0.5340212	total: 527ms	remaining: 404ms
566:	learn: 0.5338027	total: 528ms	remaining: 403ms
567:	learn: 0.5336349	total: 529ms	remaining: 403ms
568:	learn: 0.5335067	total: 531ms	remaining: 402ms
569:	learn: 0.5333512	total: 532ms	remaining: 401ms
570:	learn: 0.5330621	total: 532ms	remaining: 400ms
571:	learn: 0.5329896	total: 533ms	remaining: 399ms
572:	learn: 0.5328995	total: 533ms	remaining: 397ms
573:	learn: 0.5327977	total: 534ms	remaining: 396ms
574:	learn: 0.5326517	total: 535ms	remaining: 395ms
575:	learn: 0.5324796	total: 536ms	remaining: 394ms
576:	learn: 0.5323341	total: 537ms	remaining: 393ms
577:	learn: 0.5322643	total: 537ms	remaining: 392ms
578:	learn: 0.5322004	total: 538ms	remaining: 391ms
579:	learn: 0.5321227	total: 539ms	remaining: 390ms
580:	learn: 0.5320640	total: 540ms	remaining: 389ms
581:	learn: 0.5318872	total: 541ms	remaining: 389ms
582:	learn: 0.5317392	total: 542ms	remaining: 388ms
583:	learn: 

821:	learn: 0.4983030	total: 707ms	remaining: 153ms
822:	learn: 0.4982138	total: 709ms	remaining: 152ms
823:	learn: 0.4980976	total: 710ms	remaining: 152ms
824:	learn: 0.4979608	total: 710ms	remaining: 151ms
825:	learn: 0.4978207	total: 711ms	remaining: 150ms
826:	learn: 0.4975700	total: 712ms	remaining: 149ms
827:	learn: 0.4974407	total: 713ms	remaining: 148ms
828:	learn: 0.4973031	total: 713ms	remaining: 147ms
829:	learn: 0.4972019	total: 714ms	remaining: 146ms
830:	learn: 0.4971685	total: 715ms	remaining: 145ms
831:	learn: 0.4969820	total: 715ms	remaining: 144ms
832:	learn: 0.4968922	total: 716ms	remaining: 144ms
833:	learn: 0.4968912	total: 716ms	remaining: 143ms
834:	learn: 0.4966940	total: 717ms	remaining: 142ms
835:	learn: 0.4966085	total: 718ms	remaining: 141ms
836:	learn: 0.4965474	total: 719ms	remaining: 140ms
837:	learn: 0.4965079	total: 720ms	remaining: 139ms
838:	learn: 0.4963977	total: 721ms	remaining: 138ms
839:	learn: 0.4962653	total: 721ms	remaining: 137ms
840:	learn: 

...

958:	learn: 0.2702476	test: 0.9650679	best: 0.6823910 (23)	total: 1.12s	remaining: 47.8ms
959:	learn: 0.2700260	test: 0.9650537	best: 0.6823910 (23)	total: 1.12s	remaining: 46.7ms
960:	learn: 0.2697475	test: 0.9653430	best: 0.6823910 (23)	total: 1.12s	remaining: 45.5ms
961:	learn: 0.2695017	test: 0.9648200	best: 0.6823910 (23)	total: 1.12s	remaining: 44.3ms
962:	learn: 0.2692106	test: 0.9647809	best: 0.6823910 (23)	total: 1.12s	remaining: 43.2ms
963:	learn: 0.2690316	test: 0.9649015	best: 0.6823910 (23)	total: 1.12s	remaining: 42ms
964:	learn: 0.2689766	test: 0.9650663	best: 0.6823910 (23)	total: 1.13s	remaining: 40.8ms
965:	learn: 0.2688658	test: 0.9650519	best: 0.6823910 (23)	total: 1.13s	remaining: 39.6ms
966:	learn: 0.2688124	test: 0.9652543	best: 0.6823910 (23)	total: 1.13s	remaining: 38.5ms
967:	learn: 0.2685849	test: 0.9651572	best: 0.6823910 (23)	total: 1.13s	remaining: 37.5ms
968:	learn: 0.2684366	test: 0.9649613	best: 0.6823910 (23)	total: 1.13s	remaining: 36.3ms
969:	learn: 

60:	learn: 6.1849316	test: 5.6540411	best: 5.6024644 (16)	total: 96.4ms	remaining: 1.48s
61:	learn: 6.1722309	test: 5.6578764	best: 5.6024644 (16)	total: 97.7ms	remaining: 1.48s
62:	learn: 6.1631050	test: 5.6590902	best: 5.6024644 (16)	total: 102ms	remaining: 1.52s
63:	learn: 6.1519191	test: 5.6591276	best: 5.6024644 (16)	total: 103ms	remaining: 1.51s
64:	learn: 6.1333749	test: 5.6613055	best: 5.6024644 (16)	total: 104ms	remaining: 1.5s
65:	learn: 6.1188574	test: 5.6670363	best: 5.6024644 (16)	total: 105ms	remaining: 1.48s
66:	learn: 6.1107894	test: 5.6614619	best: 5.6024644 (16)	total: 106ms	remaining: 1.47s
67:	learn: 6.1085079	test: 5.6631666	best: 5.6024644 (16)	total: 106ms	remaining: 1.46s
68:	learn: 6.0970677	test: 5.6660860	best: 5.6024644 (16)	total: 107ms	remaining: 1.44s
69:	learn: 6.0953896	test: 5.6669654	best: 5.6024644 (16)	total: 111ms	remaining: 1.48s
70:	learn: 6.0896829	test: 5.6628267	best: 5.6024644 (16)	total: 114ms	remaining: 1.49s
71:	learn: 6.0852432	test: 5.66

202:	learn: 5.0649592	test: 6.0811466	best: 5.6024644 (16)	total: 241ms	remaining: 948ms
203:	learn: 5.0636643	test: 6.0825990	best: 5.6024644 (16)	total: 244ms	remaining: 952ms
204:	learn: 5.0631031	test: 6.0826982	best: 5.6024644 (16)	total: 245ms	remaining: 949ms
205:	learn: 5.0560928	test: 6.0796002	best: 5.6024644 (16)	total: 245ms	remaining: 946ms
206:	learn: 5.0553624	test: 6.0799721	best: 5.6024644 (16)	total: 246ms	remaining: 942ms
207:	learn: 5.0517792	test: 6.0822446	best: 5.6024644 (16)	total: 246ms	remaining: 938ms
208:	learn: 5.0492829	test: 6.0828373	best: 5.6024644 (16)	total: 247ms	remaining: 934ms
209:	learn: 5.0449152	test: 6.0811366	best: 5.6024644 (16)	total: 247ms	remaining: 930ms
210:	learn: 5.0365381	test: 6.0791571	best: 5.6024644 (16)	total: 248ms	remaining: 926ms
211:	learn: 5.0304836	test: 6.0793445	best: 5.6024644 (16)	total: 248ms	remaining: 922ms
212:	learn: 5.0257571	test: 6.0830864	best: 5.6024644 (16)	total: 249ms	remaining: 919ms
213:	learn: 5.0233685

561:	learn: 3.8224896	test: 6.7038268	best: 5.6024644 (16)	total: 405ms	remaining: 316ms
562:	learn: 3.8208676	test: 6.7056076	best: 5.6024644 (16)	total: 406ms	remaining: 315ms
563:	learn: 3.8185841	test: 6.7063032	best: 5.6024644 (16)	total: 407ms	remaining: 314ms
564:	learn: 3.8169337	test: 6.7064768	best: 5.6024644 (16)	total: 407ms	remaining: 313ms
565:	learn: 3.8151233	test: 6.7054912	best: 5.6024644 (16)	total: 407ms	remaining: 312ms
566:	learn: 3.8132660	test: 6.7035810	best: 5.6024644 (16)	total: 408ms	remaining: 312ms
567:	learn: 3.8125789	test: 6.7037724	best: 5.6024644 (16)	total: 409ms	remaining: 311ms
568:	learn: 3.8104106	test: 6.7067466	best: 5.6024644 (16)	total: 409ms	remaining: 310ms
569:	learn: 3.8088436	test: 6.7074116	best: 5.6024644 (16)	total: 410ms	remaining: 309ms
570:	learn: 3.8069001	test: 6.7110005	best: 5.6024644 (16)	total: 410ms	remaining: 308ms
571:	learn: 3.8066539	test: 6.7111952	best: 5.6024644 (16)	total: 411ms	remaining: 307ms
572:	learn: 3.8056423

850:	learn: 3.4816302	test: 6.8691097	best: 5.6024644 (16)	total: 566ms	remaining: 99.1ms
851:	learn: 3.4814922	test: 6.8691064	best: 5.6024644 (16)	total: 567ms	remaining: 98.4ms
852:	learn: 3.4804125	test: 6.8723271	best: 5.6024644 (16)	total: 567ms	remaining: 97.7ms
853:	learn: 3.4799293	test: 6.8723477	best: 5.6024644 (16)	total: 569ms	remaining: 97.2ms
854:	learn: 3.4793971	test: 6.8740691	best: 5.6024644 (16)	total: 569ms	remaining: 96.6ms
855:	learn: 3.4792961	test: 6.8742474	best: 5.6024644 (16)	total: 570ms	remaining: 95.9ms
856:	learn: 3.4786506	test: 6.8741242	best: 5.6024644 (16)	total: 571ms	remaining: 95.2ms
857:	learn: 3.4779431	test: 6.8734814	best: 5.6024644 (16)	total: 571ms	remaining: 94.6ms
858:	learn: 3.4774464	test: 6.8738143	best: 5.6024644 (16)	total: 572ms	remaining: 93.9ms
859:	learn: 3.4771521	test: 6.8736356	best: 5.6024644 (16)	total: 573ms	remaining: 93.2ms
860:	learn: 3.4763357	test: 6.8747731	best: 5.6024644 (16)	total: 573ms	remaining: 92.5ms
861:	learn

In [102]:
from prettytable import PrettyTable
from PIL import Image, ImageDraw, ImageFont
table = PrettyTable()
table.field_names = ['Var', 'OLS','DML-L','DML-Boost','DML-NN']
a = ['θ_hat']+ np.mean(MC_θ, axis = 0).tolist()
table.add_row(a)
a = ['First Stage Y R2']+ np.mean(MC_y, axis = 0).tolist()
table.add_row(a)
a = ['First Stage D Accuracy']+ np.mean(MC_t, axis = 0).tolist()
table.add_row(a)
table.float_format = '0.3'
print(table)

+------------------------+-------+--------+-----------+--------+
|          Var           |  OLS  | DML-L  | DML-Boost | DML-NN |
+------------------------+-------+--------+-----------+--------+
|         θ_hat          | 1.683 | 1.765  |   1.838   | 2.011  |
|    First Stage Y R2    | 0.039 | -0.007 |   -0.011  | -0.215 |
| First Stage D Accuracy | 0.589 | 0.582  |   0.627   | 0.560  |
+------------------------+-------+--------+-----------+--------+


# First Stage

In [57]:
model_Lasso = LinearDML(discrete_treatment=True, random_state=1, cv=2)
model_Lasso.fit(y.ravel(), t.ravel(), X=None,W=x)
θ_DMLL = model_Lasso.intercept_

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


In [141]:
# Checking Logistic regression
import numpy as np
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,t, test_size=0.3, random_state = 42)
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape) 
model = LogisticRegression()
model.fit(x_train, y_train)
preds_class = model.predict(x_test)
preds_proba = model.predict_proba(x_test)
from sklearn.metrics import roc_auc_score, accuracy_score, log_loss, f1_score
log_loss(y_test, preds_proba[:,0]), roc_auc_score(y_test, preds_proba[:,0]), accuracy_score(y_test, preds_class), f1_score(y_test, preds_class)

(364, 1) (364,) (157, 1) (157,)


(0.714211406639855, 0.4621732026143791, 0.5414012738853503, 0.7024793388429752)

In [156]:
# Checking Catboost

import numpy as np
from catboost import CatBoostClassifier, Pool, metrics
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x,t, test_size=0.3)
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape) 

# initialize data
train_data = x_train
train_labels = y_train
test_data = catboost_pool = Pool(x_test,y_test)
model = CatBoostClassifier(iterations=10000,
                           depth=8,
                           learning_rate=0.01,
                           loss_function='Logloss',
                           verbose=1000, 
                           eval_metric="Accuracy",
                           eval_fraction=0.2)
model.fit(x_train, y_train)

(364, 1) (364,) (157, 1) (157,)
0:	learn: 0.5993151	test: 0.5342466	best: 0.5342466 (0)	total: 1.26ms	remaining: 12.6s
1000:	learn: 0.7191781	test: 0.5479452	best: 0.6027397 (7)	total: 907ms	remaining: 8.15s
2000:	learn: 0.7363014	test: 0.5753425	best: 0.6027397 (7)	total: 1.99s	remaining: 7.96s
3000:	learn: 0.7397260	test: 0.5616438	best: 0.6027397 (7)	total: 3.04s	remaining: 7.09s
4000:	learn: 0.7397260	test: 0.5616438	best: 0.6027397 (7)	total: 4.09s	remaining: 6.13s
5000:	learn: 0.7397260	test: 0.5616438	best: 0.6027397 (7)	total: 5.06s	remaining: 5.06s
6000:	learn: 0.7397260	test: 0.5616438	best: 0.6027397 (7)	total: 6.17s	remaining: 4.11s
7000:	learn: 0.7397260	test: 0.5616438	best: 0.6027397 (7)	total: 7.22s	remaining: 3.09s
8000:	learn: 0.7397260	test: 0.5616438	best: 0.6027397 (7)	total: 8.26s	remaining: 2.06s
9000:	learn: 0.7397260	test: 0.5616438	best: 0.6027397 (7)	total: 9.26s	remaining: 1.03s
9999:	learn: 0.7397260	test: 0.5616438	best: 0.6027397 (7)	total: 10.3s	remainin

In [171]:
# Checking Catboost

import numpy as np
from catboost import CatBoostClassifier, Pool, metrics
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.3)
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape) 

# initialize data
train_data = x_train
train_labels = y_train
test_data = catboost_pool = Pool(x_test,y_test)
model = CatBoostRegressor(iterations=10000,
                           depth=12,
                           learning_rate=0.01,
                           loss_function='RMSE',
                           verbose=100,
                           eval_metric="R2",
                           eval_fraction=0.2)
model.fit(x_train, y_train)

(364, 1) (364,) (157, 1) (157,)
0:	learn: 0.0075058	test: 0.0024301	best: 0.0024301 (0)	total: 630us	remaining: 6.3s
100:	learn: 0.4075502	test: 0.3125693	best: 0.3125693 (100)	total: 394ms	remaining: 38.6s
200:	learn: 0.4970583	test: 0.3757928	best: 0.3757928 (200)	total: 753ms	remaining: 36.7s
300:	learn: 0.5270238	test: 0.3940882	best: 0.3940910 (299)	total: 1.27s	remaining: 41.1s
400:	learn: 0.5442462	test: 0.4024884	best: 0.4024884 (400)	total: 1.67s	remaining: 40s
500:	learn: 0.5570542	test: 0.4042267	best: 0.4042267 (500)	total: 2.06s	remaining: 39s
600:	learn: 0.5685458	test: 0.4076423	best: 0.4076423 (600)	total: 2.47s	remaining: 38.6s
700:	learn: 0.5807441	test: 0.4108128	best: 0.4108128 (700)	total: 2.89s	remaining: 38.4s
800:	learn: 0.5919856	test: 0.4128935	best: 0.4128935 (800)	total: 3.32s	remaining: 38.1s
900:	learn: 0.6019592	test: 0.4156672	best: 0.4159072 (892)	total: 3.79s	remaining: 38.3s
1000:	learn: 0.6099588	test: 0.4165554	best: 0.4167026 (980)	total: 4.26s	rem

KeyboardInterrupt: 